In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
df_train = pd.read_csv('/home/alvaro/tf_templates/DATA/DailyDelhiClimateTrain.csv', index_col='date', parse_dates=True)
df_test = pd.read_csv('/home/alvaro/tf_templates/DATA/DailyDelhiClimateTest.csv', index_col='date', parse_dates=True)

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
train = df_train[['meantemp']]
test = df_test[['meantemp']]

In [ ]:
train.head()

In [ ]:
train.plot(figsize=(12, 6));

In [ ]:
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train)
scaled_test = scaler.transform(test)

In [ ]:
length = 30
batch_size = 32
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=batch_size)
validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size=batch_size)

In [ ]:
n_features = 1

In [ ]:
model = Sequential([
    Input(shape=(length, n_features)),
    LSTM(units=100, activation='relu'),
    Dense(units=1)
])

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
model.fit(generator, epochs=100, validation_data=validation_generator, callbacks=[early_stop])

In [ ]:
history = pd.DataFrame(model.history.history)

In [ ]:
history[['loss', 'val_loss']].plot();

In [ ]:
test_predictions = []
first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(scaled_test)):
    current_prediction = model.predict(current_batch)[0]
    test_predictions.append(current_prediction)
    current_batch = np.append(current_batch[:, 1:, :], [[current_prediction]], axis=1)

In [ ]:
predictions = scaler.inverse_transform(test_predictions)
test.loc[:, 'Predictions'] = predictions

In [ ]:
test.plot();